In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
import pandas as pd
import sys
#%pip install --disable-pip-version-check pycaret
sys.path.append("../src")
from funcoes import *
from funcoes_ import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

###1- Train & Test

####1.1 - General Checks

In [0]:
#Após recebermos uma base enriquecida precisamos dividir entre Treino e Teste, para evitar Data Leakage.
#Data Leakage é uma falha que acontece durante os testes de um modelo de machine learning,
#no qual informações são compartilhadas entre um conjunto de dados usado no treinamento
#e outro conjunto de dados que é usado para validar essa modelagem, também chamado de dataset de testes.
#

In [0]:
base_modelagem = pd.read_csv("../data/processed/base_modelagem.csv", sep=",")

In [0]:
base_modelagem.head()

In [0]:
def perfil_base_conversao(
    base_modelo: pd.DataFrame,
    id_col: str,
    target_col: str,
    safra_col: str
) -> dict:
    """
    Calcula métricas básicas do perfil da base de dados focada em conversão.

    Parâmetros:
    - base_modelo (pd.DataFrame): DataFrame contendo os dados a serem analisados.
    - id_col (str): Nome da coluna que representa o identificador único (ID).
    - target_col (str): Nome da coluna que representa a variável alvo (1 = convertido, 0 = não convertido).
    - safra_col (str): Nome da coluna que representa a safra.

    Retorna:
    - dict: Dicionário contendo:
        - shape: Tupla com a quantidade de linhas e colunas.
        - tipos_variaveis: Contagem dos tipos das variáveis.
        - ids_unicos: Quantidade de IDs únicos.
        - taxa_conversao: Taxa de conversão da base.
        - volumetria_safras: Quantidade de registros por safra.
    """
    perfil = {}

    # 1. Shape da base
    perfil['shape'] = f"Essa base possui {base_modelo.shape[0]} linhas e {base_modelo.shape[1]} colunas"

    # 2. Tipos das variáveis
    perfil['tipos_variaveis'] = base_modelo.dtypes.value_counts().to_dict()

    # 3. IDs únicos
    perfil['ids_unicos'] = base_modelo[id_col].nunique()

    # 4. Taxa de conversão
    if target_col in base_modelo.columns:
        total_conversoes = base_modelo[target_col].value_counts().to_dict()

        # Evitar KeyError se faltar alguma das classes
        nao_convertidos = total_conversoes.get(0, 0)
        convertidos = total_conversoes.get(1, 0)

        taxa_conv = convertidos / (nao_convertidos + convertidos) if (nao_convertidos + convertidos) > 0 else 0

        perfil['taxa_conversao'] = {
            "não_convertidos": nao_convertidos,
            "convertidos": convertidos,
            "taxa_percentual": round(taxa_conv * 100, 2)
        }
    else:
        perfil['taxa_conversao'] = "Coluna alvo não encontrada."

    # 5. Volumetria por safra
    if safra_col in base_modelo.columns:
        perfil['volumetria_safras'] = dict(
            sorted(base_modelo[safra_col].value_counts().to_dict().items())
        )
    else:
        perfil['volumetria_safras'] = "Coluna safra não encontrada."

    # Prints amigáveis
    print("📊 Perfil da base de dados (Conversão)")
    print(f"Shape da base: {perfil['shape']}")
    print(f"Tipos de variáveis: {perfil['tipos_variaveis']}")
    print(f"IDs únicos: {perfil['ids_unicos']}")
    if isinstance(perfil['taxa_conversao'], dict):
        print(f"Taxa de conversão: {perfil['taxa_conversao']['taxa_percentual']}% "
              f"({perfil['taxa_conversao']['convertidos']} convertidos, "
              f"{perfil['taxa_conversao']['não_convertidos']} não convertidos)")
    else:
        print(f"Taxa de conversão: {perfil['taxa_conversao']}")
    print(f"Volumetria das safras: {perfil['volumetria_safras']}")
    print("\n")

    return perfil


In [0]:
#A função perfil_base auxilia na verificação de métricas básicas da base de dados
resultado = perfil_base_conversao(base_modelagem, id_col='ID', target_col='target_sucesso', safra_col='time_since_test_start')

In [0]:
from typing import Optional
def plot_safra_conversion_rate(
    df: pd.DataFrame,
    safra_col: str = "safra",
    conversao_col: str = "y",
    conv_rate_min: Optional[float] = None,
    conv_rate_max: Optional[float] = None
) -> pd.DataFrame:
    """
    Gera um gráfico de barras com a contagem por safra e
    uma linha com a taxa de conversão no eixo secundário.

    Retorna:
    - DataFrame com: safra, contagem, total_convertidos, total_nao_convertidos, conversion_rate.
    """
    import matplotlib.pyplot as plt
    from typing import Optional
    import pandas as pd

    # Garantir que safra seja numérica para ordenação
    df[safra_col] = pd.to_numeric(df[safra_col], errors="coerce")

    # Agrupar e ordenar
    safra_stats = (
        df.groupby(safra_col)
        .agg(
            contagem=(conversao_col, "count"),
            total_convertidos=(conversao_col, "sum")
        )
        .reset_index()
        .sort_values(safra_col)
    )

    # Calcular não convertidos e taxa
    safra_stats["total_nao_convertidos"] = safra_stats["contagem"] - safra_stats["total_convertidos"]
    safra_stats["conversion_rate"] = safra_stats["total_convertidos"] / safra_stats["contagem"]

    # Criar gráfico
    fig, ax1 = plt.subplots(figsize=(10, 5))

    # Barras - total
    ax1.bar(safra_stats[safra_col].astype(str), safra_stats["contagem"],
            color="blue", alpha=0.6, label="Total")
    ax1.set_xlabel("Safra")
    ax1.set_ylabel("Total de IDs", color="blue")
    ax1.tick_params(axis="y", labelcolor="blue")
    ax1.set_xticklabels(safra_stats[safra_col].astype(str), rotation=45)

    # Linha - taxa de conversão
    ax2 = ax1.twinx()
    ax2.plot(safra_stats[safra_col].astype(str), safra_stats["conversion_rate"],
             color="green", marker="o", linestyle="-", linewidth=2, label="Taxa de Conversão")
    ax2.set_ylabel("Taxa de Conversão (%)", color="green")
    ax2.tick_params(axis="y", labelcolor="green")

    # Limites opcionais do eixo secundário
    if conv_rate_min is not None and conv_rate_max is not None:
        ax2.set_ylim(conv_rate_min, conv_rate_max)

    plt.title("Total por Safra e Taxa de Conversão")
    fig.tight_layout()
    plt.show()

    return safra_stats


In [0]:
base_modelagem["time_since_test_start"] = base_modelagem["time_since_test_start"].astype(float).astype("int")
safra_br = plot_safra_conversion_rate(base_modelagem, safra_col="time_since_test_start", conversao_col="target_sucesso", conv_rate_min=0, conv_rate_max=0.8)

In [0]:
safra_br

In [0]:
#verificando a tipagem das variáveis
base_modelagem.dtypes.to_frame().value_counts(0)

In [0]:
#verificando a quantidade de safras e ofertas
base_modelagem[['time_since_test_start','offer_id']].value_counts().to_frame().reset_index().sort_values('time_since_test_start')


####1.2 - Train Test Division

In [0]:
#Apesar da visão de "safras" vamos resolver esse problema apenas comm Treino e Teste OOS
#Isso porque estamos falando de apenas um mes de campanha com novos disparos consecutivos
#As variáveis nao devem perder poder preditivo em um período de tempo tão curto.

In [0]:
base_modelagem.shape

In [0]:
#A base de treino deve ser dividida em Desenvolvimento e validação (train test)
#Para que possamos avaliar os modelos dentro do mesmo contexto temporal do treinamento OOS
# A base -> treino precisa ser dividida em train, test oos (testar a performance nas safras conhecidas)
# 30% da base é o teste oos.
train, test_oos = train_test_split(base_modelagem, test_size=0.3, stratify=base_modelagem['target_sucesso'], random_state=42)
train.shape, test_oos.shape

In [0]:
#verificando as taxas de conversao
train['target_sucesso'].value_counts()/len(train)*100

In [0]:
test_oos['target_sucesso'].value_counts()/len(test_oos)*100

In [0]:
#Salvando as bases. A partir daqui todas as análises e transformacoes serao realizadas no conjunto de treino
#E aplicada quando necessário no conjunto de testes
train.to_csv("../data/processed/train.csv",sep=",",index=False,header=True)
test_oos.to_csv("../data/processed/test_oos.csv",sep="|",index=False,header=True)

### 2 - Exploratory Data Analysis (EDA)

In [0]:
train= pd.read_csv("../data/processed/train.csv",sep=",")

In [0]:
train.head()

In [0]:
#Analisar os dados!! Quais variáveis vamos manter? descartar? transformar ou criar? Quais os Insights?
target = "target_sucesso"

####2.1 - Types

In [0]:
target = "target_sucesso"

num_cols, cat_cols = detectar_tipos(
    train,
    target=target,
    exclude_cols=["ID","cliente_id","offer_id","time_since_test_start"],  # não classificar esses
    force_cat=["genero","offer_type","safra_registro","recebeu_email","recebeu_mobile","recebeu_social","recebeu_web",
        "social","web","mobile","email"],                   # garantir que fiquem categóricas
    force_num=[
        "canais_recebidos_total","discount_value","duration","min_value",
        "n_instancias_anteriores_mesma_oferta","qtd_ofertas_completas_validas",
        
    ],
    int_low_card_as_cat=False  # não transformar ints de baixa cardinalidade em categorias
)

print("Categóricas:", cat_cols)
print("Numéricas (amostra):", num_cols[:10], " ... total:", len(num_cols))


In [0]:
num_cols

In [0]:
cat_cols

In [0]:
for col in cat_cols:
    if col in train.columns:
        train[col] = train[col].astype('category')

In [0]:
train.dtypes

In [0]:
tipagem = train.dtypes.reset_index()
dicionario_tipagem = dict(zip(tipagem["index"], tipagem[0]))

In [0]:
dicionario_tipagem

####2.2 - Target

In [0]:
target_balance(train, target)

####2.3 - Missing

In [0]:
#avaliacao rapida de missings na base, posteriormente vamos filtrar os campos com mais de 40% de missing
drop_missing = ["dias_desde_ultima_conversao_valida_mesma",
"dias_desde_ultima_oferta_mesma",
"n_conversoes_validas_anteriores_mesma",
"tempo_medio_conversao",
"dias_desde_ultima_conversao_valida",
"tempo_medio_visualizacao"]

resumo_missing(train).head(20)

####2.4 - Variables

In [0]:
num_cols

In [0]:
#Essas variáveis possuem problemas de inconsistencia devemos rever o conceito no book
drop_inconsistencia = ["mobile_ratio_historico","social_ratio_historico","dias_desde_ultima_conversao_valida_mesma","mobile_ratio_historico","social_ratio_historico","tempo_medio_conversao","web_ratio_historico","dias_desde_ultima_transacao.1"]

resumo_numericas(train, num_cols)

In [0]:
#p_valor baixo, vamos manter todas
chi2_categoricas(train, cat_cols, target)

In [0]:
# 5) Correlação + pares fortes
corr, pares_fortes = correlacao_numericas(train, num_cols)
display(pares_fortes)

In [0]:
#remover variaveis com alto colinearidade
drop_alta_colinearidade = pares_fortes["var2"].tolist()

In [0]:
drop_alta_colinearidade

In [0]:
# 7) WoE/IV (ranking)
#IV/WOE → mede capacidade de discriminação da variável para um problema binário.
#A variável ["n_conversoes_validas_anteriores_mesma"] tem um IV absurdo provavelmente algum problema na construcao da variavel (leakeage)
#Devemos remover ou consertar o conceito
#Vale lembrar que estamos falando de correlacoes lineares, portanto nao vamos nos livrar 
#de variaveis que apenas tem iv baixo.
drop_iv_absurdo = ["n_conversoes_validas_anteriores_mesma"]
display(woe_iv_todas(train, num_cols + cat_cols, target, n_bins=10, metodo="quantile"))

In [0]:
drop_missing

In [0]:
remover_vars = list(set(drop_inconsistencia+drop_missing+drop_alta_colinearidade+drop_iv_absurdo))

In [0]:
resumo_missing(train)

In [0]:
train.columns

In [0]:
train.dtypes

In [0]:
remover_vars

In [0]:
vars_sobreviventes = [x for x in train if x not in remover_vars]

In [0]:
vars_sobreviventes

In [0]:
#Sobraram 49 variáveis!!
len(vars_sobreviventes)

In [0]:
train_selecionado = train[vars_sobreviventes]

In [0]:
ids_ =["ID","time_since_test_start","offer_id","cliente_id"]
target="target_sucesso"

In [0]:
#correlacao maxima de 0.8
matriz_correlacao(train_selecionado.drop(ids_,axis=1).iloc[:, 1:15])

###3 - Data Prep

In [0]:
#Nesta fase seguimos garantindo a qualidade dos dados que serao utilizados para treinar nosso modelo
#Nesta fase seguimos garantindo a qualidade dos dados que serao utilizados para treinar nosso modelo

In [0]:
#Nesta fase seguimos garantindo a qualidade dos dados que serao utilizados para treinar nosso modelo

####3.1 - Handling Missing Values

In [0]:
a# Nesta etapa importante vamos verificar nossas variáveis e propor um tratamento adequado para os valores faltantes:

In [0]:
train_selecionado.head()

In [0]:
train_selecionado.dtypes

In [0]:
missing_plan_df = build_missing_plan()
missing_plan_df

In [0]:
train_selecionado

In [0]:
#A variável idade é um tanto complicada, nao sabemos o real motivo dela estar faltando mas vamos 
#partir da premissa que se trata de um missing aleatório.
#Para tentar capturar quando o missing ocorreu vamos criar uma flag_missing, para que o modelo tenha chance de
#aprender quando o missing ocorreu.
train_selecionado["idade_missing"] = train_selecionado["idade"].isna().astype(int)

In [0]:
train_selecionado["idade"].count()

In [0]:
train_selecionado["idade_missing"].value_counts()

In [0]:
#Definindo o plano de imputação
plan = build_missing_plan()

prep = fit_missing_prep(train_selecionado, plan)
train_tratado = apply_missing_prep(train_selecionado, prep, copy=True)

# Para o teste:
# test_tratado = apply_missing_prep(test, prep, copy=True)


In [0]:
train_tratado.head()

In [0]:
#As variáveis de oferta permaneceram, muito bom para a nossa estratégia de escoragem
#variar as ofertas para o cliente e verificar maior probabilidade de conversao!
variaveis_de_oferta = ["offer_id","offer_type","discount_value","min_value","duration","web","email","mobile","social"]

In [0]:

#Alguns modelos são sensíveis a magnitude, das variáveis, mas não é o caso do modelo de lightGBM que vamos treinar
#Sem padronizar os dados.
#Sem remover outlier pois o modelo tambem nao é sensível a outliers
#Agora podemos seguir em paz
resumo_missing(train_tratado).head(20)